In [ ]:
FORGE_INSTANCE_URL = "https://v0yf0nginj87vx-1234.proxy.runpod.net" # "http://127.0.0.1:1234" / "https://*-1234.proxy.runpod.net"

In [ ]:
import numpy as np
import requests
import base64
import json
import time
import cv2
import os
os.makedirs("_input", exist_ok=True)
os.makedirs("_output", exist_ok=True)

In [ ]:
def imagen_upscale(img_inp_path, img_out_path, scale_factor=4, upscaler="R-ESRGAN 4x+"):
    # ----- UPSCALE -----
    # R-ESRGAN 4x+ / R-ESRGAN 4x+ Anime6B
    # -------------------
    print(f"UPSCALE > {img_inp_path} > {upscaler} > x{scale_factor}")

    with open(img_inp_path, "rb") as f_img_inp:
        img_inp_base64 = base64.b64encode(f_img_inp.read()).decode('utf-8')
    # img_h, img_w, _ = cv2.imread(img_inp_path).shape

    payload = {
        "resize_mode": 0,                         # Sets the resize mode: 0 to upscale by upscaling_resize amount, 1 to upscale up to upscaling_resize_h x upscaling_resize_w.
        "show_extras_results": False,             # Should the backend return the generated image?
        "gfpgan_visibility": 0.35,                # Sets the visibility of GFPGAN, values should be between 0 and 1.
        "codeformer_visibility": 0,               # Sets the visibility of CodeFormer, values should be between 0 and 1.
        "codeformer_weight": 0,                   # Sets the weight of CodeFormer, values should be between 0 and 1.
        "upscaling_resize": scale_factor,         # By how much to upscale the image, only used when resize_mode=0.
        "upscaling_resize_w": 512,                # Target width for the upscaler to hit. Only used when resize_mode=1.
        "upscaling_resize_h": 512,                # Target height for the upscaler to hit. Only used when resize_mode=1.
        "upscaling_crop": True,                   # Should the upscaler crop the image to fit in the chosen size?
        "upscaler_1": upscaler,                   # The name of the main upscaler to use
        "upscaler_2": "None",                     # The name of the secondary upscaler to use
        "extras_upscaler_2_visibility": 0,        # Sets the visibility of secondary upscaler, values should be between 0 and 1.
        "upscale_first": False,                   # Should the upscaler run before restoring faces?
        "image": img_inp_base64                   # Image to work on, must be a Base64 string containing the image's data.
    }

    def request_forge_instance(payload, img_out_path):
        try:
            with requests.post(url=FORGE_INSTANCE_URL+"/sdapi/v1/extra-single-image", json=payload) as req:
                with open(img_out_path, "wb") as f:
                    f.write(base64.b64decode(req.json()["image"]))
        except Exception as e:
            print(f"⚠️ imagen_upscale (x{scale_factor}) > Error: {e}")
            print(f"⚠️ imagen_upscale (x{scale_factor}) > req.json(): {req.json()}")
    request_forge_instance(payload, img_out_path)

def imagen_img2img(img_inp_path, img_msk_path=None, img_out_path="_output/_test.jpg", sd_params={}):
    # ----- IMG2IMG -----
    # 1. Mask -> Inpaint mask area
    # 2. No mask -> Inpaint the whole image
    # -------------------
    # sd_params = {
    #     "model": "WAI_V11",
    #     "prompt": f"masterpiece, best quality, <PROMPT>",
    #     "negative": "bad quality, worst quality, worst detail",
    #     "sampler": "Euler a",
    #     "scheduler": "Karras",
    #     "steps": 20,
    #     "cfg": 6.0,
    #     "denoising": 0.6,
    #     "seed": -1,
    # }
    # -------------------
    print(f"IMG2IMG > {img_inp_path} > {sd_params}")
    
    img_inp_base64 = None
    img_msk_base64 = None
    if img_inp_path:
        with open(img_inp_path, "rb") as f:
            img_inp_base64 = base64.b64encode(f.read()).decode('utf-8')
    if img_msk_path:
        with open(img_msk_path, "rb") as f:
            img_msk_base64 = base64.b64encode(f.read()).decode('utf-8')
    img_h, img_w, _ = cv2.imread(img_inp_path).shape

    payload = {
        "prompt": sd_params["prompt"],
        "negative_prompt": sd_params["negative"],
        "styles": [],
        "seed": sd_params["seed"],
        "subseed": -1,
        "subseed_strength": 0,
        "seed_resize_from_h": -1,
        "seed_resize_from_w": -1,
        "sampler_name": sd_params["sampler"],
        "scheduler": sd_params["scheduler"],
        "batch_size": 1,
        "n_iter": 1,
        "steps": sd_params["steps"],
        "cfg_scale": sd_params["cfg"],
        "distilled_cfg_scale": 3.5,
        "width": img_w,
        "height": img_h,
        "restore_faces": False,
        "tiling": False,
        "do_not_save_samples": False,
        "do_not_save_grid": False,
        "eta": 0,
        "denoising_strength": sd_params["denoising"],
        "s_min_uncond": 0.0,
        "s_churn": 0.0,
        "s_tmax": None,
        "s_tmin": 0.0,
        "s_noise": 1.0,
        "override_settings": {"sd_model_checkpoint": sd_params["model"], "CLIP_stop_at_last_layers": 1},
        "override_settings_restore_afterwards": False,
        # "refiner_checkpoint": "string",
        "refiner_switch_at": 0,
        "disable_extra_networks": False,
        # "firstpass_image": "string",
        "comments": {},
        "init_images": [img_inp_base64],
        "resize_mode": 1,
        "image_cfg_scale": 1.5,
        "mask": img_msk_base64,
        "mask_blur_x": 4,
        "mask_blur_y": 4,
        "mask_blur": 4,
        "mask_round": True,
        "inpainting_fill": 1,
        "inpaint_full_res": 0,
        "inpaint_full_res_padding": 32,
        "inpainting_mask_invert": 0,
        "initial_noise_multiplier": 1.0,
        # "latent_mask": "string",
        # "force_task_id": "string",
        "hr_distilled_cfg": 3.5,
        "sampler_index": "Euler",
        "include_init_images": False,
        "script_name": None,
        "script_args": [],
        "send_images": True,
        "save_images": False,
        "alwayson_scripts": {},
        # "infotext": "string"
    }

    def request_forge_instance(payload, savepath):
        try:
            with requests.post(url=FORGE_INSTANCE_URL+"/sdapi/v1/img2img", json=payload) as req:
                
                # Optional - Add seed info to filename
                actual_seed = json.loads(req.json()["info"])["seed"]
                savepath = savepath.replace(".jpg", f" ({actual_seed}).jpg")

                with open(savepath, "wb") as f:
                    f.write(base64.b64decode(req.json()["images"][0]))
        except Exception as e:
            print(f"⚠️ imagen_img2img > Error: {e}")
            print(f"⚠️ imagen_img2img > req.json(): {req.json()}")
    request_forge_instance(payload, img_out_path)

# # Test both imagen_img2img and imagen_upscale
# SD_PARAMS = {
#     "model": "WAI_V11",
#     "prompt": f"masterpiece, best quality, artist: wamudraws",
#     "negative": "bad quality, worst quality, worst detail",
#     "sampler": "Euler a",
#     "scheduler": "Karras",
#     "steps": 20,
#     "cfg": 6.0,
#     "denoising": 0.6,
#     "seed": -1,
# }
# img_inp_path = "_input/_test.jpg"
# imagen_img2img(img_inp_path, img_msk_path=None, img_out_path="_output/_test_img2img.jpg", sd_params=SD_PARAMS)
# imagen_upscale(img_inp_path, img_out_path="_output/_test_upscale.jpg", scale_factor=4, upscaler="R-ESRGAN 4x+ Anime6B")

In [ ]:
def imagen_facemask(img_inp_path, img_keypoints_path, img_mask_path, expand_scale_factor=1.5):
    # -------------------- Helpers --------------------
    def helper_expand_polygon(ls_keypoints, expand_scale_factor):
        points = np.array(ls_keypoints, dtype=np.float32)                      # Convert keypoints to a NumPy array
        centroid = np.mean(points, axis=0)                                     # Calculate the centroid of the polygon
        expanded_points = centroid + (points - centroid) * expand_scale_factor # Translate points to origin (subtract centroid), scale, and translate back
        expanded_points = expanded_points.astype(np.int32)                     # Convert back to integer coordinates for OpenCV
        return expanded_points.tolist()
    def helper_create_mask_image(img_inp_path, img_mask_path, ls_keypoints):
        h, w, _ = cv2.imread(img_inp_path).shape
        img_mask = np.full((h, w, 3), 255, dtype=np.uint8) # Create a white mask (Default to inpaint all)
        if len(ls_keypoints) == 0:
            print(f"⚠️ imagen_facemask > {img_inp_path} > No face detected")
        elif len(ls_keypoints) == 1:
            img_mask = np.full((h, w, 3), 0, dtype=np.uint8) # Create a black mask
            cv2.fillPoly(img_mask, [np.array(ls_keypoints[0], np.int32)], (255, 255, 255)) # Fill the face polygon with white
        elif len(ls_keypoints) > 1:
            print(f"⚠️ imagen_facemask > {img_inp_path} > Many faces detected")
        cv2.imwrite(img_mask_path, img_mask)
    def helper_draw_keypoints(img_inp_path, img_keypoints_path, ls_keypoints, ls_keypoints_expanded):
        img = cv2.imread(img_inp_path)
        LINE_THICKNESS = int(img.shape[1]/200)
        POINT_THICKNESS = int(img.shape[1]/150)
        for i in range(len(ls_keypoints)):
            cv2.polylines(img, [np.array(ls_keypoints[i], dtype=np.int32)], isClosed=True, color=(0, 0, 255), thickness=LINE_THICKNESS)
            cv2.polylines(img, [np.array(ls_keypoints_expanded[i], dtype=np.int32)], isClosed=True, color=(0, 255, 0), thickness=LINE_THICKNESS)
        for ls_kps in ls_keypoints:
            for (x, y) in ls_kps:
                cv2.circle(img, (x, y), radius=POINT_THICKNESS, color=(0, 255, 0), thickness=-1)
        cv2.imwrite(img_keypoints_path, img)
    # -------------------- Extract ls_keypoints --------------------
    def get_ls_keypoints_mediapipe(img_inp_path):
        import mediapipe as mp
        ls_keypoints = []
        try:
            MEDIAPIPE_FACE_OUTLINE_IDXS = [10,338,297,332,284,251,389,356,454,323,361,288,397,365,379,378,400,377,152,148,176,149,150,136,172,58,132,93,234,127,162,21,54,103,67,109]
            img = cv2.imread(img_inp_path)
            with mp.solutions.face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True) as face_mesh:
                results = face_mesh.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
                if results.multi_face_landmarks:
                    for facelandmarks in results.multi_face_landmarks:
                        h, w, _ = img.shape
                        ls_kps = [(int(facelandmarks.landmark[idx].x*w), int(facelandmarks.landmark[idx].y*h)) for idx in MEDIAPIPE_FACE_OUTLINE_IDXS]
                        ls_keypoints.append(ls_kps)
        except Exception as er:
            print(f"⚠️ Error > {er}")
        return ls_keypoints
    def get_ls_keypoints_deepface(img_inp_path):
        from deepface import DeepFace
        ls_keypoints = []
        try:
            face_objs = []
            try: face_objs = DeepFace.extract_faces(img_inp_path, detector_backend='retinaface', align=True)
            except: pass
            for fce in face_objs:
                x,y,w,h = fce["facial_area"]["x"], fce["facial_area"]["y"], fce["facial_area"]["w"], fce["facial_area"]["h"]
                ls_keypoints.append([(x,y), (x,y+h), (x+w,y+h), (x+w,y)])
        except Exception as er:
            print(f"⚠️ Error > {er}")
        return ls_keypoints
    # -------------------- Main --------------------
    ls_keypoints = get_ls_keypoints_mediapipe(img_inp_path)
    if len(ls_keypoints) == 0:
        ls_keypoints = get_ls_keypoints_deepface(img_inp_path)
    ls_keypoints_expanded = [helper_expand_polygon(e, expand_scale_factor) for e in ls_keypoints]
    helper_draw_keypoints(img_inp_path, img_keypoints_path, ls_keypoints, ls_keypoints_expanded)
    helper_create_mask_image(img_inp_path, img_mask_path, ls_keypoints_expanded)

# imagen_facemask(img_inp_path="_input/_test.jpg", img_keypoints_path="_output/_test_facemask_kps.jpg", img_mask_path="_output/_test_facemask.jpg", expand_scale_factor=1.5)

-----
# Inference
-----

In [ ]:
# ARTIST_TAGS = ['','akairiot','aogisa','bacun','chengongzi123','chihunhentai','ciloranko','donburi_(donburikazoku)','dramus','eigaka','eu03','ge-b','gogalking','haoni','horn/wood','hu_dako','hxd','inudori','jima','kidmo','kotorai','kyuuba_melo','linsun','love_cacao','menma_(enaic31)','milkychu','minaba_hideo','mochizuki_kei','nac000','noriuma','oastlv','pottsness','rariatto_(ganguri)','ratatatat74','sho_(sho_lwlw)','siu_(siu0207)','slugbox','starshadowmagician','tianliang_duohe_fangdongye','varniskarnis','wamudraws','zankuro']
# ARTIST_TAGS = ['','aogisa','bacun','chengongzi123','ciloranko','donburi_(donburikazoku)','eigaka','eu03','ge-b','gogalking','haoni','inudori','jima','kidmo','kyuuba_melo','linsun','love_cacao','milkychu','minaba_hideo','mochizuki_kei','nac000','noriuma','oastlv','rariatto_(ganguri)','ratatatat74','sho_(sho_lwlw)','siu_(siu0207)','starshadowmagician','tianliang_duohe_fangdongye','varniskarnis','wamudraws','zankuro']
# ARTIST_TAGS = ['chengongzi123', 'oastlv', 'ciloranko','tianliang_duohe_fangdongye','kyuuba_melo','starshadowmagician']
# ARTIST_TAGS = ['chengongzi123','ciloranko','oastlv','tianliang_duohe_fangdongye']
# ARTIST_TAGS = ['chengongzi123, ciloranko, oastlv, tianliang_duohe_fangdongye']
# ARTIST_TAGS = ['chengongzi123, ciloranko, tianliang_duohe_fangdongye', 'ciloranko, tianliang_duohe_fangdongye, chengongzi123', 'tianliang_duohe_fangdongye, chengongzi123, ciloranko']

for artist_tag in ARTIST_TAGS:
    SD_PARAMS = {
        "model": "WAI_V11",
        
        
        
        "prompt": f"({artist_tag}:0.9), general, (1boy, portrait, looking_at_viewer:1.1), (masterpiece, best quality, amazing quality:0.5)",
        


        "negative": "bad quality, worst quality, worst detail",
        "sampler": "Euler a",
        "scheduler": "Karras",
        "steps": 20,
        "cfg": 7.0,
        "denoising": 1.0,

        # "seed": -1,
        # "seed": 1234567890,
        # "seed": 9876543210,
        # "seed": 2468135790,
        # "seed": 1357924680,
    }
    img_inp_path = "_input/1792x1024.jpg"
    img_out_path = "_output/" + SD_PARAMS["prompt"].replace(":", "").replace("/", "") + " (" + SD_PARAMS["model"] + ")" + ".jpg"
    # ----------
    time_start = time.time()
    try:
        imagen_img2img(img_inp_path=img_inp_path, img_msk_path=None, img_out_path=img_out_path, sd_params=SD_PARAMS)
    except Exception as er:
        try:
            imagen_img2img(img_inp_path=img_inp_path, img_msk_path=None, img_out_path=img_out_path, sd_params=SD_PARAMS)
        except Exception as er:
            print(f"⚠️ Error: {er}")
    time_end = time.time()
    print(f"Execution time: {time_end - time_start:.1f} seconds")